In [197]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns

# read data

In [198]:
#แก้
df_fix_point = pd.read_csv('data/ANP2.2_Default_fixed_point_profiles.csv',delimiter=';')
df_fix_point = df_fix_point[df_fix_point.ACFT_ID == "727200"]
df_fix_point = df_fix_point[df_fix_point['Stage Length'] == 1]
df_fix_point = df_fix_point.reset_index(drop=True)
df_fix_poit_initial = df_fix_point[df_fix_point['Op Type'] == "A"]
# df_fix_poit_initial

In [199]:
#แก้
df = {'AIQ311-1670716320-schedule-0614':{},'AIQ512-1670716320-schedule-0609':{}}

In [200]:
#----- read flight data -----#
for key in df.keys():
    df[key]['df'] = pd.read_csv(f'ONLY_TEST/{key}.csv')
    df[key]['df'].drop(columns=['Unnamed: 0'],inplace=True)


# add power setting

In [201]:
def add_powersetting_2_flight(df_param):
    df__tmp = df_fix_poit_initial.copy().groupby('TAS (kt)').max()
    df__tmp = df__tmp.reindex(df__tmp.index.union(df_param.groundspeed.unique()))
    df__tmp = df__tmp.interpolate(method='index',limit_direction='both',limit=200)
    df__tmp = df__tmp.reindex(df_param.groundspeed.unique())
    df__tmp.reset_index(drop=False, inplace=True)
    df__tmp.rename(columns={'TAS (kt)':'groundspeed'}, inplace=True)
    df__tmp = df__tmp[['groundspeed', 'Power Setting']]

    df_param = df_param.merge(df__tmp, how='left', on='groundspeed')
    
    return df_param

In [202]:
for key in df.keys():
    df[key]['df'] = add_powersetting_2_flight(df[key]['df'])


# creat npd data for flight

In [203]:
npd = pd.read_csv('data/ANP2.2_NPD_data.csv',delimiter=';')
# npd.head()

In [204]:
def get_feet(distance):
    feet = distance.split('_')[1]
    return float(feet[:-2])

In [205]:
#แก้
def npd_inter_per_flight(df_param):
    npd_of_flight = npd[npd['NPD_ID'] == 'CF567B'] #<-----ตรงนี้
    npd_of_flight = npd_of_flight[npd_of_flight['Noise Metric'] == 'LAmax']
    npd_of_flight = npd_of_flight[npd_of_flight['Op Mode'] == 'A']
    npd_of_flight.set_index('Power Setting', inplace=True)

    npd_of_flight = npd_of_flight.reindex(npd_of_flight.index.union(df_param['Power Setting'].unique()))
    npd_of_flight = npd_of_flight.interpolate(method='index',limit_direction='both',limit=200)
    
    npd_of_flight = npd_of_flight.drop(columns=['NPD_ID','Noise Metric','Op Mode']).T.reset_index()
    npd_of_flight.rename(columns={'index':'distance'},inplace=True)

    npd_of_flight['distance'] = npd_of_flight['distance'].apply(get_feet)
    npd_of_flight.set_index('distance',inplace=True)
    return npd_of_flight

In [206]:
for key in df.keys():
    df[key]['npd'] = npd_inter_per_flight(df[key]['df'])

# cal grid

In [207]:
#----- calculate distance from lat, long -----#
def add_distance(lat1, long1, lat2, long2):
    position1 = (lat1,long1)
    position2 = (lat2,long2)
    return geopy.distance.geodesic(position1, position2).ft

In [208]:
#----- function Single Soung Event -----#
def function_L(sound) :
    return 10**(sound / 10)

In [209]:
# 13.911475, 100.605997 DMK
# 50 ช่อง -> เพิ่มทีละ 0.004
100.505997, 100.705997
13.811475, 14.011475

(13.811475, 14.011475)

In [210]:
lat_start =  13.811475
long_start = 100.505997
L_dB_start = 0

def create_observerer():
    global lat_start, long_start, L_dB_start
    observer = np.zeros((5,5,3))
    for i in range(5) :
        for j in range(5) :
            observer[i][j] = lat_start, long_start, L_dB_start
            long_start += 0.004
        lat_start  += 0.004
        long_start = 100.505997           
    return observer

In [211]:
for key in df.keys():
    df[key]['grid'] = create_observerer()

In [212]:
def Calculate_Grid(observer,df_param, npd_param):
    for i in range(len(observer)):
        for j in range(len(observer[i])): # j = [lat, long, L(dB)]
            #----- copy df_param -----#
            tmp = df_param.copy()
            
            #----- calculate distance every transection -----#
            tmp_distance = []
            tmp['distance'] = -1
            
            for index, row in tmp.iterrows():
                distance = add_distance(observer[i][j][0], observer[i][j][1], row['latitude'], row['longitude'])
                tmp_distance.append(distance)
                tmp.loc[index, 'distance'] = distance
            
            #----- calculate L_dB from powersetting & distance -> copy to model_data -----#
            npd_after_inter = npd_param.reindex(npd_param.index.union(tmp_distance))
            npd_after_inter = npd_after_inter.interpolate(method='index',limit_direction='both',limit=200)
            model_data = npd_after_inter.copy()
            
            #----- add L_dB to df['sound'] & select < 25000 only -----#
            for index, row in tmp.iterrows():
                tmp.loc[index,'sound'] = model_data.loc[row.distance,row['Power Setting']]
                
            tmp = tmp[tmp['distance']<25000]
            
            #----- Single Sound Event Model -----#
            if tmp.empty:
                continue
            tmp['Function'] = tmp['sound'].apply(function_L)
            area = integrate.simpson(tmp['Function'])
            lSEL= 10*np.log10( (1/1)*area)
            
            #----- add l_dB to observer -----#
            observer[i][j][2] = lSEL
            
    return observer

In [213]:
for key in df.keys():
    df[key]['grid'] = Calculate_Grid(df[key]['grid'], df[key]['df'], df[key]['npd'])

In [214]:
#----- observer to pivot table -----#
for key in df.keys():
    df_tmp = pd.DataFrame(columns = ['Lat','Long','L_dB'])
    
    #----- matrix to pandas -----#
    for i in range(len(df[key]['grid'])):
        row = pd.DataFrame(df[key]['grid'][i], columns = ['Lat','Long','L_dB'])
        df_tmp = pd.concat([df_tmp, row])
    
    #----- Create pivot table -----#
    tabal_df = pd.pivot_table(df_tmp,index='Lat',columns='Long',values='L_dB')
    df[key]['grid'] = tabal_df

# cumulative

In [215]:
#----- result grid for plot -----#
res_grid = np.zeros((5,5))

In [216]:
#----- Variable Value for Cumulative Level ( Day-Night ) -----#
Day_Delta_i = 0
Night_Delta_i = 10
t0 = 1
T0 = 86400

In [217]:
#----- function Cumulative Level ( Day-Night ) -----#
def Cumulative_model(DN, sound):
    global Day_Delta_i, Night_Delta_i
    if DN == 'night':
        return 10**( (sound + Night_Delta_i) / 10 )
    elif DN == 'day':
        return 10**( (sound + Day_Delta_i) / 10 )
    else:
        return 0

In [218]:
#แก้
for key in df.keys():
    df[key]['period'] = 'day'

In [219]:
for i in range(len(df[list(df.keys())[0]]['grid'])):
    for j in range(len(df[list(df.keys())[0]]['grid'])):
        LDN = 0
        for key in df.keys():
            LDN = LDN + Cumulative_model(df[key]['period'], df[key]['grid'].iloc[i, j])
        res_grid[i][j] = 10*np.log10( (t0 / T0) * LDN )

In [220]:
res_grid

array([[19.2291968 , 18.45539813, 17.82014155, 17.28061475, 16.75984413],
       [22.04075013, 21.17660888, 20.36944407, 19.61607674, 18.91697628],
       [25.94812493, 24.6737253 , 23.53481837, 22.57143266, 21.77746876],
       [31.53490163, 29.39686401, 27.99872945, 26.76911485, 25.5331047 ],
       [45.85547463, 39.48719335, 35.42491726, 32.88053747, 30.67542472]])